
# Assignment 3


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = 'drive/MyDrive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
'''
from detectron2.structures import BoxMode
import math, random

def get_detection_data(set_name):

  data_dirs = '{}/data'.format(BASE_DIR)
  if set_name == "train.json":
    file = os.path.join(data_dirs, set_name)
    with open(file) as f:
      jfile = json.load(f)
    dataset = {}
    for idx, v in enumerate(jfile):
      filename = os.path.join(data_dirs, "train", v["file_name"])
      anno = v["segmentation"]
      px = [anno[0][i] for i in range(0, len(anno[0]), 2)]
      py = [anno[0][i] for i in range(1, len(anno[0]), 2)]
      poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
      poly = [p for x in poly for p in x]
      obj = {
          "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
          "bbox_mode": BoxMode.XYXY_ABS,
          "segmentation": [poly],
          "category_id": 0,
          }
      if not filename in dataset:
        dataset[filename] = {}
        height, width = cv2.imread(filename).shape[:2]
        dataset[filename]['file_name'] = filename
        dataset[filename]['image_id'] = idx
        dataset[filename]['height'] = height
        dataset[filename]['width'] = width
        dataset[filename]['annotations'] = []
      dataset[filename]['annotations'].append(obj)
    dataset = np.array(list(dataset.values()))
  elif set_name == "test.json":
    dataset = []
    for file in os.listdir(data_dirs + '/test'):
      record = {}
      filename = os.path.join(data_dirs, "test", file)

      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["height"] = height
      record["width"] = width
      record["annotations"] = []
      dataset.append(record)
  return dataset

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "test"]:
  DatasetCatalog.register("data_detection_" + d, lambda d = d:get_detection_data(d + ".json"))
  MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("data_detection_train")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset = get_detection_data("train.json")
for d in random.sample(list(dataset), 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_metadata, scale=0.40)
    out = visualizer.draw_dataset_dict(d)
    print(d["file_name"])
    cv2_imshow(out.get_image()[:, :, ::-1])

### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 


### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
from detectron2.engine import DefaultTrainer
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/CMPT_CV_lab3/output

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)


In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("test.json")
count = 0
for d in dataset_dicts:
  if count == 3:
    break    
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.4, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])
  count+=1

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", output_dir=OUTPUT_DIR)
train_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(predictor.model, train_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''
#inc epochsto 1000
#Batch_size = 32



## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''

def get_instance_sample(data, idx, img=None):
  img = cv2.imread(data["file_name"])
  anno = data['annotations'][idx]
  bbox = anno['bbox']
  minX,minY,maxX,maxY = bbox
  obj_img = img[minY:maxY, minX:maxX]
  seg = anno["segmentation"][0]
  obj_mask = detectron2.utils.visualizer.GenericMask(data["annotations"][idx]["segmentation"], data['height'], data['width']).mask
  obj_mask = obj_mask[minY:maxY, minX:maxX]
  return obj_img, obj_mask

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    img, mask = get_instance_sample(data,idx[1])
    #resize
    img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA) 
    mask = cv2.resize(mask, (128, 128), interpolation = cv2.INTER_AREA)
    img, mask = self.numpy_to_tensor(img, mask) 
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# Conv block module 
'''
class block(nn.Module):
    def __init__(self, ch):
      super(block, self).__init__()
      self.layer = nn.Sequential(
          nn.Conv2d(ch, ch, 3, padding=1),
          nn.BatchNorm2d(ch),
          nn.ReLU(inplace=True),
          nn.Conv2d(ch, ch, 3, padding=1),
          nn.BatchNorm2d(ch),
          nn.ReLU(inplace=True)
      )
    def forward(self, x):
      x = self.layer(x)
      return x
'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # pseudo-resnet + decoder
        
        # Encoder
        
        self.input_conv = conv(3, 16)
        self.block1 = block(16)
        self.down1 = down(16, 32)
        self.block2 = block(32)
        self.down2 = down(32, 64)
        self.block3 = block(64)
        self.down3 = down(64, 128)
        self.block4 = block(128)
        self.down4 = down(128, 256)
        self.block5 = block(256)
        self.down5 = down(256, 512)
        self.block6 = block(512)
        self.down6 = down(512, 1024)
        
        # Decoder
        
        self.up1 = up(1024, 512)
        self.up2 = up(512, 256)
        self.up3 = up(256, 128)
        self.up4 = up(128, 64)
        self.up5 = up(64, 32)
        self.up6 = up(32, 16)
        self.up7 = conv(16, 3)
        self.output_conv = conv(3, 1, False) # ReLu activation is removed to keep the logits for the loss function
        
        # normalisation

        self.normal1 = nn.BatchNorm2d(3)
        self.normal2 = nn.BatchNorm2d(16)
        self.normal3 = nn.BatchNorm2d(32)
        self.normal4 = nn.BatchNorm2d(64)
        self.normal5 = nn.BatchNorm2d(128)
        self.normal6 = nn.BatchNorm2d(256)
        self.normal7 = nn.BatchNorm2d(512)

    def forward(self, input):
      y = self.input_conv(input)#16

      y1 = self.block1(y)
      y2 = self.down1(y1) #32
      y2 = self.block2(y2)
      y3 = self.down2(y2) #64
      y3 = self.block3(y3)
      y4 = self.down3(y3) #128
      y4 = self.block4(y4)
      y5 = self.down4(y4) #256
      y5 = self.block5(y5)
      y6 = self.down5(y5) #512
      y6 = self.block6(y6)
      y7 = self.down6(y6) #1024

      y7 = self.normal7(F.relu(self.up1(y7) + y6)) #512
      y6 = self.normal6(F.relu(self.up2(y7) + y5)) #256
      y5 = self.normal5(F.relu(self.up3(y6) + y4)) #128
      y4 = self.normal4(F.relu(self.up4(y5) + y3)) #64
      y3 = self.normal3(F.relu(self.up5(y4) + y2)) #32
      y2 = self.normal2(F.relu(self.up6(y3) + y1)) #16
      y = self.normal1(F.relu(self.up7(y2))) #3

      output = self.output_conv(y)
      return output

In [ ]:
from torchsummary import summary
model = MyModel().cuda()
summary(model, (3, 128, 128))

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 5
batch_size = 64
learning_rate = 0.001
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as adam
# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    mask = torch.unsqueeze(mask,1)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))

In [ ]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''

def iou_coef(y_true, y_pred):
  intersection = np.logical_and(y_true, y_pred).sum()
  union = np.logical_or(y_true, y_pred).sum()
  if union == 0:
    return 0
  else:
    return intersection/union

batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
img_count = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    mask = torch.unsqueeze(mask,1).cpu().detach()
    pred = model(img).cpu().detach()
    for i in range(img.shape[0]):
      y_pred = pred[i]
      y_pred = np.rint(np.array(sigmoid(y_pred)[0]))
      y_true = mask[i]
      y_true = np.rint(np.array(sigmoid(y_true)[0]))
      total_iou = total_iou + iou_coef(y_true,y_pred)
      img_count = img_count + 1
    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''

print("\n #images: {}, Mean IoU: {}".format(img_count, total_iou/img_count))


In [ ]:
#Inverse transformations
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
batch_size = 3
loader, dataset = get_plane_dataset('train', batch_size)
count = 0
for (img, mask) in tqdm(loader):
  if count==3:
    break
  img = img.cuda()
  mask = mask.cpu().detach()
  pred = model(img).cpu().detach()
  for i in range(batch_size):
    pimg = img[i].cpu()
    pimg = invTrans(pimg)
    pimg = transforms.ToPILImage()(pimg)
    cv2_imshow(np.array(pimg))
    ppred = np.rint(np.array(sigmoid(pred[i])[0]))*255
    cv2_imshow(ppred)
    pmask = np.array(mask[i])*255
    cv2_imshow(pmask)
  count+=1

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))

In [ ]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

def get_prediction_mask(data):
  img = cv2.imread(data['file_name'])
  preds = predictor(img)['instances']
  pred_mask = np.zeros([data['height'],data['width']])
  gt_mask = np.zeros([data['height'],data['width']])
  if len(data['annotations'])==0:
    #get pred mask
    for i in range(len(preds)):
      pred_box = np.array(np.floor(list(preds[i]._fields['pred_boxes'])[0].cpu().numpy()), dtype=np.uint32)
      minX, minY, maxX, maxY = pred_box
      pred = img[minY:maxY, minX:maxX]

      pred = cv2.resize(pred, (128, 128), interpolation = cv2.INTER_AREA)
      pred = model(torch.unsqueeze(torch.tensor(transforms.ToTensor()(pred), device=torch.device('cuda')),0)).cpu().detach().numpy()[0]
      pred = cv2.resize(pred[0], (maxX-minX, maxY-minY), interpolation = cv2.INTER_AREA)
      pred = np.rint(np.array(sigmoid(pred)))*(i+1)
   
      maskpred = pred_mask[minY:maxY, minX:maxX] 
      maskpred[maskpred == 0] = 10000

      overlap = np.minimum(maskpred, pred)
      overlap[overlap == 1000] = 0
      pred_mask[minY:maxY, minX:maxX] = overlap
    pred_mask = torch.tensor(pred_mask, device=torch.device('cuda'))
    gt_mask = torch.tensor(gt_mask,device=torch.device('cuda'))
  else:
    for idx, v in enumerate(data['annotations']):
      bbox = v['bbox']
      minX,minY,maxX,maxY = bbox
      
      pred = img[minY:maxY, minX:maxX]

      pred = cv2.resize(pred, (128, 128), interpolation = cv2.INTER_AREA)
      ppred = model(torch.unsqueeze(torch.tensor(transforms.ToTensor()(pred), device=torch.device('cuda')),0)).cpu().detach().numpy()[0]
      ppred = cv2.resize(ppred[0], (maxX-minX, maxY-minY), interpolation = cv2.INTER_AREA)
      ppred = np.rint(np.array(sigmoid(ppred)))*(idx+1)

      maskpred = pred_mask[minY:maxY, minX:maxX] 
      maskpred[maskpred == 0] = 10000

      overlap = np.minimum(maskpred, ppred)
      overlap[overlap == 1000] = 0
      pred_mask[minY:maxY, minX:maxX] = overlap
      
      maxY = maxY + minY
      maxX = maxX + minX
      gmask = detectron2.utils.visualizer.GenericMask(v['segmentation'], data['height'], data['width']).mask
      overlap = np.maximum(gt_mask[minY:maxY, minX:maxX],gmask[minY:maxY, minX:maxX]*(idx+1))
      gt_mask[minY:maxY, minX:maxX] = overlap
    pred_mask = torch.tensor(pred_mask, device=torch.device('cuda'))
    gt_mask = torch.tensor(gt_mask,device=torch.device('cuda'))
  return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
dataset = get_detection_data('train.json')
for i in range(3):
  img, gt_mask, pred_mask = get_prediction_mask(dataset[i])
  cv2_imshow(cv2.resize(img, (img.shape[1]//3, img.shape[0]//3), interpolation = cv2.INTER_AREA))
  pred_mask = pred_mask.cpu().numpy()
  gt_mask = gt_mask.cpu().numpy()
  predmax = pred_mask.max()
  mpfact = 255./predmax
  pred_mask*= mpfact
  cv2_imshow(cv2.resize(gt_mask, (gt_mask.shape[1]//3, gt_mask.shape[0]//3), interpolation = cv2.INTER_AREA))
  gtmax = gt_mask.max()
  mgfact = 255./gtmax
  gt_mask*= mgfact
  cv2_imshow(cv2.resize(pred_mask, (pred_mask.shape[1]//3, pred_mask.shape[0]//3), interpolation = cv2.INTER_AREA))

In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
for d in ["train", "test"]:
  DatasetCatalog.register("mask_rcnn_" + d, lambda d = d:get_detection_data(d + ".json"))
  MetadataCatalog.get("mask_rcnn_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("mask_rcnn_train")

### Network

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("mask_rcnn_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### Training

In [ ]:
from detectron2.engine import DefaultTrainer
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/CMPT_CV_lab3/output

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("mask_rcnn_train", output_dir=OUTPUT_DIR)
train_loader = build_detection_test_loader(cfg, "mask_rcnn_train")
print(inference_on_dataset(predictor.model, train_loader, evaluator))

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("test.json")
count = 0
for d in dataset_dicts:
  if count == 3:
    break    
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.4, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])
  count+=1